In [ ]:

!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


**Summary of the Workflow**

1.Set up the LLM (HuggingFacePipeline for text generation).

2.Create a retriever (vectordb.as_retriever(k=1)) to fetch the most relevant document.

3.Build a question-answering chain (RetrievalQA.from_chain_type).

4.Define a structured prompt that instructs the model not to hallucinate.

5.Apply the custom prompt to the QA pipeline.

In [ ]:
import torch
from transformers import AutoTokenizer,pipeline,AutoModelForSeq2SeqLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
#setup_embeddings
def create_sbert_mpnet():
   model_name ='sentence-transformers/all-mpnet-base-v2'
   device = "cuda" if torch.cuda.is_available() else "cpu"
   return HuggingFaceEmbeddings(model_name = model_name, model_kwargs={"device":device})

def create_flan_t5_base(load_in_8bit):
  #wrap it in HF pipeline for use with LangChain
  model = "google/flan-t5-xl"
  tokenizer = AutoTokenizer.from_pretrained(model)
  return pipeline(
      task = "text2text-generation",
      model=model,
      tokenizer=tokenizer,
      max_new_tokens=100,
      device_map = "auto",
      model_kwargs = {"load_in_8bit": load_in_8bit, "temperature" : 0.95}

  )

embedding = create_sbert_mpnet()
llm = create_flan_t5_base(load_in_8bit=False)

<ipython-input-6-4b0c22c3b0b2>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name = model_name, model_kwargs={"device":device})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or data

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


**Use LLM to ask Q without RAG**

In [ ]:
question = "what is computer science?"

prompt = "Answer the following question. \n" + question

res = llm(prompt)

print(res)

[{'generated_text': 'computer science is the study of the theory and application of computer technology'}]


**RAG framework**

In [ ]:
pdf_path = "/content/computer_science_is_foundational.pdf"
loader = PDFPlumberLoader(pdf_path)
docs = loader.load()

## split docs and create text snippets
#text_splitter = CharacterTextSplitter()
# texts = text_splitte.split_documents(docs)

text_splitter = TokenTextSplitter(chunk_size=300, chunk_overlap=10)
texts = text_splitter.split_documents(docs)

persist_directory = "./chromadb"
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


**Print chunks of extracted texts**

In [ ]:
print(len(texts))

5


In [ ]:
for t in texts[:5]:
  print(t)
  print("##**##"*20)


page_content='What is Computer Science and What do People Do
Once They Know It?
What Is Computer Science?
Computer science develops students’ computational and critical thinking skills and shows them how to
create, not simply use, new technologies. This fundamental knowledge is needed to prepare students
for the 21st century, regardless of their ultimatefield of study or occupation.
Additional Background: Computer science education encompasses “the study of computers
and algorithmic processes, including their principles, their hardware and software designs, their
applications, and their impact on society.” A few of the topics and activities that might be included in a
computer science course include:
• Algorithmic problem- solving
• Computing and data analysis (managing, processing, visualizing and interpreting data)
• Human- computer interaction
• Modeling and simulating real- world problems
• Creating and manipulating graphics
• Programming (including game design)
• Security (includi

In [ ]:
vdb = vectordb.get(include = ['embeddings', 'documents', 'metadatas'])
num=2

print(vdb['documents'][:num])
print(vdb['embeddings'][:num])
print(vdb['metadatas'][:num])

['What is Computer Science and What do People Do\nOnce They Know It?\nWhat Is Computer Science?\nComputer science develops students’ computational and critical thinking skills and shows them how to\ncreate, not simply use, new technologies. This fundamental knowledge is needed to prepare students\nfor the 21st century, regardless of their ultimatefield of study or occupation.\nAdditional Background: Computer science education encompasses “the study of computers\nand algorithmic processes, including their principles, their hardware and software designs, their\napplications, and their impact on society.” A few of the topics and activities that might be included in a\ncomputer science course include:\n• Algorithmic problem- solving\n• Computing and data analysis (managing, processing, visualizing and interpreting data)\n• Human- computer interaction\n• Modeling and simulating real- world problems\n• Creating and manipulating graphics\n• Programming (including game design)\n• Security (inc

**create RAG framework**

In [ ]:
##In a RAG pipeline, we need to fetch relevant documents from a knowledge base before passing them to
## the LLM. The retriever is responsible for this step.

In [ ]:
hf_llm = HuggingFacePipeline(pipeline=llm)

#Returns the closest document that is connected to the user prompt.k = 1 document
#as_retriever() converts the vector database into a retriever object that can fetch the most relevant documents.
retriever = vectordb.as_retriever(search_kwargs = {"k":1})

##RetrievalQA is a LangChain module that performs question answering using retrieved documents.
##from_chain_type creates a standard question-answering chain.
#stuff: "stuff" means that all retrieved documents are simply
## "stuffed" into the prompt before passing them to the LLM.

##Other chain types include "map_reduce" (for summarization) and "refine" (iterative refinement).
qa = RetrievalQA.from_chain_type(llm=hf_llm, chain_type="stuff", retriever=retriever)

# Defining a default prompt for flan models
##The model is instructed not to hallucinate and should only answer based on the provided context
question_t5_template = """
 I am a helpful AI that answers questions.
 When I don't know the answer I say I don't know.
I know context :{context}
when asked : {question}
my response using only information in the context is:
"""
prompt = PromptTemplate(
        template = question_t5_template, input_variables=["context", "question"]
    )

##qa.combine_documents_chain refers to the part of the chain that combines retrieved documents.
qa.combine_documents_chain.llm_chain.prompt = prompt



<ipython-input-19-fa72a00702cb>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(pipeline=llm)


In [ ]:
question = "what is computer science?"

##verbose = True makes the chain print detailed logs of how it processes inputs.
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query":question,})

<ipython-input-21-bde1c6380c3a>:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa({"query":question,})




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': 'what is computer science?',
 'result': 'the study of computers and algorithmic processes, including their principles, their hardware and software designs, their applications, and their impact on society',
 'source_documents': [Document(metadata={'CreationDate': "D:20130912171604-07'00'", 'Creator': 'Adobe InDesign CC (Macintosh)', 'ModDate': "D:20130912171605-07'00'", 'Producer': 'Adobe PDF Library 10.0.1', 'Trapped': 'False', 'file_path': '/content/computer_science_is_foundational.pdf', 'page': 0, 'source': '/content/computer_science_is_foundational.pdf', 'total_pages': 2}, page_content='What is Computer Science and What do People Do\nOnce They Know It?\nWhat Is Computer Science?\nComputer science develops students’ computational and critical thinking skills and shows them how to\ncreate, not simply use, new technologies. This fundamental knowledge is needed to prepare students\nfor the 21st century, regardless of their ultimatefield of study or occupation.\nAdditional Back